Create NetCDF from monthly precipitation data

In [30]:
import os
import numpy as np
import rasterio
from netCDF4 import Dataset, date2num
from datetime import datetime, timedelta

# Define file pattern and date range for monthly data
file_pattern = 'PRISM_ppt_stable_4kmM3_{}_bil.bil'
start_date = datetime(1981, 1, 1)
end_date = datetime(2022, 12, 1)

# Define the extent and resolution
extent = (-125.0208333333335, 24.0625, -66.4791666661985, 49.9375)
lon_start, lat_start, lon_end, lat_end = extent
lon_res = 0.04166666666667  # 4km resolution in degrees
lat_res = 0.04166666666667

# Generate latitude and longitude arrays based on the actual data dimensions
nlat, nlon = 621, 1405  # Actual dimensions from the sample data
lon = np.linspace(lon_start, lon_end, nlon)
lat = np.linspace(lat_end, lat_start, nlat)

# Create the NetCDF file
output_file = 'PRISM_monthly_precipitation_1981_2022.nc'
if os.path.exists(output_file):
    os.remove(output_file)

nco = Dataset(output_file, 'w', format='NETCDF4')

# Create dimensions
nco.createDimension('lat', nlat)
nco.createDimension('lon', nlon)
nco.createDimension('time', None)

# Create variables
lato = nco.createVariable('lat', 'f4', ('lat',))
lono = nco.createVariable('lon', 'f4', ('lon',))
timeo = nco.createVariable('time', 'f4', ('time',))
preco = nco.createVariable('precipitation', 'f4', ('time', 'lat', 'lon',), fill_value=np.nan)  # Set fill value to NaN

# Set variable attributes
timeo.units = 'days since 1858-11-17 00:00:00'
timeo.calendar = 'gregorian'
timeo.standard_name = 'time'
lato.units = 'degrees_north'
lato.standard_name = 'latitude'
lono.units = 'degrees_east'
lono.standard_name = 'longitude'
preco.units = 'mm/month'
preco.standard_name = 'monthly precipitation'

# Assign coordinate variables
lato[:] = lat
lono[:] = lon

# Initialize time array
months = []
current_month = start_date
while current_month <= end_date:
    months.append(current_month)
    current_month = (current_month.replace(day=28) + timedelta(days=4)).replace(day=1)  # Move to the first day of the next month

total_months = len(months)
time_values = date2num(months, units=timeo.units, calendar=timeo.calendar)
timeo[:] = time_values

# Read and assign precipitation data
for month, current_month in enumerate(months):
    file_name = file_pattern.format(current_month.strftime('%Y%m'))
    try:
        with rasterio.open(file_name) as src:
            data = src.read(1).astype(np.float32)  # Ensure data is in float32 format to handle NaNs
            if data.shape != (nlat, nlon):
                raise ValueError(f"Data shape mismatch: {data.shape} vs {(nlat, nlon)}")
            preco[month, :, :] = data
    except FileNotFoundError:
        print(f"File not found: {file_name}, filling with NaNs")
        preco[month, :, :] = np.nan
    except ValueError as e:
        print(f"Error with file {file_name}: {e}")
        preco[month, :, :] = np.nan
    if month % 12 == 0:
        print(f"Processed {month}/{total_months} months")

# Close the NetCDF file
nco.close()

print(f"NetCDF file created: {output_file}")


Processed 0/504 months
Processed 12/504 months
Processed 24/504 months
Processed 36/504 months
Processed 48/504 months
Processed 60/504 months
Processed 72/504 months
Processed 84/504 months
Processed 96/504 months
Processed 108/504 months
Processed 120/504 months
Processed 132/504 months
Processed 144/504 months
Processed 156/504 months
Processed 168/504 months
Processed 180/504 months
Processed 192/504 months
Processed 204/504 months
Processed 216/504 months
Processed 228/504 months
Processed 240/504 months
Processed 252/504 months
Processed 264/504 months
Processed 276/504 months
Processed 288/504 months
Processed 300/504 months
Processed 312/504 months
Processed 324/504 months
Processed 336/504 months
Processed 348/504 months
Processed 360/504 months
Processed 372/504 months
Processed 384/504 months
Processed 396/504 months
Processed 408/504 months
Processed 420/504 months
Processed 432/504 months
Processed 444/504 months
Processed 456/504 months
Processed 468/504 months
Processed 

In [31]:
import xarray as xr

def inspect_data(file_path):
    try:
        ds = xr.open_dataset(file_path)
        print("Dataset loaded successfully.")
        print(ds)
        print(ds.variables)
    except Exception as e:
        print(f"Error loading dataset: {e}")

file_path = 'PRISM_monthly_precipitation_1981_2022.nc'
inspect_data(file_path)

Dataset loaded successfully.
<xarray.Dataset>
Dimensions:        (lat: 621, lon: 1405, time: 504)
Coordinates:
  * lat            (lat) float32 49.94 49.9 49.85 49.81 ... 24.15 24.1 24.06
  * lon            (lon) float32 -125.0 -125.0 -124.9 ... -66.56 -66.52 -66.48
  * time           (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2022-12-01
Data variables:
    precipitation  (time, lat, lon) float32 ...
Frozen({'lat': <xarray.IndexVariable 'lat' (lat: 621)>
array([49.9375  , 49.895767, 49.85403 , ..., 24.145967, 24.104235, 24.0625  ],
      dtype=float32)
Attributes:
    units:          degrees_north
    standard_name:  latitude, 'lon': <xarray.IndexVariable 'lon' (lon: 1405)>
array([-125.020836, -124.97913 , -124.93744 , ...,  -66.56256 ,  -66.52087 ,
        -66.479164], dtype=float32)
Attributes:
    units:          degrees_east
    standard_name:  longitude, 'time': <xarray.IndexVariable 'time' (time: 504)>
array(['1981-01-01T00:00:00.000000000', '1981-02-01T00:00:00.000000000',


Calculate SPI-3 from the precipitation NetCDF

In [3]:
import xarray as xr
import numpy as np
from scipy import stats as st
import rasterio
from rasterio.transform import from_origin
import os
import pandas as pd  # Import pandas for datetime conversion

# Load the PRISM dataset
da_data = xr.open_dataset('PRISM_monthly_precipitation_1981_2022.nc')

# Select the precipitation data
ds_RR = da_data['precipitation']

# Standardized Precipitation Index Function
def spi(ds, thresh):
    # Rolling Mean / Moving Averages
    ds_ma = ds.rolling(time=thresh, center=False).mean()
    
    # Natural log of moving averages, handling zeros and negative values
    ds_ma = ds_ma.where(ds_ma > 0)
    ds_In = np.log(ds_ma)
    
    # Overall Mean of Moving Averages
    ds_mu = ds_ma.mean('time')
    
    # Summation of Natural log of moving averages
    ds_sum = ds_In.sum('time')
    
    # Computing essentials for gamma distribution
    n = ds_In[thresh-1:, :, :].count('time')  # Size of data
    A = np.log(ds_mu) - (ds_sum / n)             # Computing A
    alpha = (1 / (4 * A)) * (1 + (1 + (4 * A / 3))**0.5)  # Computing alpha (a)
    beta = ds_mu / alpha                         # Computing beta (scale)
    
    # Gamma Distribution (CDF)
    gamma_func = lambda data, a, scale: st.gamma.cdf(data, a=a, scale=scale)
    gamma = xr.apply_ufunc(gamma_func, ds_ma, alpha, beta)
    
    # Standardized Precipitation Index (Inverse of CDF)
    norminv = lambda data: st.norm.ppf(data, loc=0, scale=1)
    norm_spi = xr.apply_ufunc(norminv, gamma)  # loc is mean and scale is standard dev.
    
    return norm_spi

# Apply the SPI function with a 3-month scale
i = 3
spi_result = spi(ds_RR, i)

# Create a directory to save GeoTIFF files
output_dir = 'spi_tiffs'
os.makedirs(output_dir, exist_ok=True)

# Loop over each time step and save SPI as GeoTIFF with band names
for idx, timestamp in enumerate(spi_result.time.values):
    # Convert numpy.datetime64 to datetime
    timestamp = pd.to_datetime(str(timestamp))
    
    # Form the filename
    filename = os.path.join(output_dir, f"spi_{timestamp.strftime('%Y%m')}.tif")
    
    # Extract the SPI value for this time step
    spi_slice = spi_result.isel(time=idx)
    
    # Get data as numpy array
    spi_array = spi_slice.values
    
    # Get spatial information
    transform = from_origin(ds_RR.lon.values[0], ds_RR.lat.values[0], 
                            ds_RR.lon.values[1] - ds_RR.lon.values[0], 
                            ds_RR.lat.values[1] - ds_RR.lat.values[0])
    height, width = spi_array.shape
    
    # Write to GeoTIFF with CRS WGS84
    with rasterio.open(filename, 'w', driver='GTiff', 
                       width=width, height=height, count=1, 
                       dtype=spi_array.dtype, 
                       crs='EPSG:4326', transform=transform) as dst:
        dst.write(spi_array, 1)
        
    # Set band name
    with rasterio.open(filename, 'r+', driver='GTiff') as dst:
        dst.update_tags(b1=dict(NAME=f'SPI_{timestamp.strftime("%Y-%m")}'))

print("GeoTIFF files saved successfully.")

GeoTIFF files saved successfully.


In [1]:
#Check if the time resolution is months

import xarray as xr

# Load the PRISM dataset
da_data = xr.open_dataset('PRISM_monthly_precipitation_1981_2022.nc')

# Print the time values to verify the resolution
print(da_data['time'])

<xarray.DataArray 'time' (time: 504)>
array(['1981-01-01T00:00:00.000000000', '1981-02-01T00:00:00.000000000',
       '1981-03-01T00:00:00.000000000', ..., '2022-10-01T00:00:00.000000000',
       '2022-11-01T00:00:00.000000000', '2022-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2022-12-01
Attributes:
    standard_name:  time
No units assigned to the time dimension.
